In [ ]:
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


class CustomDataset(Dataset):
    def __init__(self, data, target="target", normalize=True):
        super(CustomDataset, self).__init__()
        self.x = data.drop(target, axis=1)

        # 데이터 표준화
        if normalize:
            scaler = StandardScaler()
            self.x = pd.DataFrame(scaler.fit_transform(self.x))

        self.y = data[target]

        # 텐서 변환
        self.x = torch.tensor(self.x.values).float()
        self.y = torch.tensor(self.y).float()

    def __len__(self):
        return len(self.x) #df 행 길이 반환

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx] #상속 문법?
        return x, y
# data 셋 상속
# init len getitem 지정 필요

1) __init__(self, data, target="target", Normalize=True): CustomDataset 클래스의 생성자 메서드입니다. 제공된 'data' 및 'target' 열 이름으로 데이터세트를 초기화합니다. 'normalize'를 'True'로 설정하면 데이터 표준화를 수행합니다.

2) super(CustomDataset, self).__init__()는 CustomDataset의 상위 클래스 생성자에 대한 호출입니다. Python에서 super()는 하위 클래스 내 상위 클래스의 메서드와 속성에 액세스하는 데 사용됩니다.
이 맥락에서:
CustomDataset은 하위 클래스입니다.
self는 하위 클래스(CustomDataset)의 인스턴스를 나타냅니다.
__init__()는 상위 클래스의 생성자 메서드입니다.

super(CustomDataset, self).__init__()를 호출하면 상위 클래스(이 경우 Dataset)의 생성자가 호출됩니다. 이는 초기화 논리를 계속 사용하면서 상위 클래스의 기능을 확장하려는 경우에 자주 사용됩니다. 이 코드 조각에서는 CustomDataset 클래스의 사용자 정의 초기화 로직 이전에 Dataset 클래스의 초기화 로직이 실행되도록 합니다.

3) 데이터 정규화:


if 정규화:: 정규화가 활성화되어 있는지 확인합니다.
scaler = StandardScaler(): scikit-learn에서 StandardScaler 인스턴스를 생성합니다.
self.x = pd.DataFrame(scaler.fit_transform(self.x)): StandardScaler의 fit_transform 메서드를 사용하여 기능(self.x)에 표준화를 적용하고 결과를 다시 변환합니다. DataFrame으로.

4) self.y = data[target]: data에서 대상 변수를 추출합니다.


5) 텐서 변환:


self.x = torch.tensor(self.x.values).float(): DataFrame 기능을 PyTorch 텐서로 변환하고 float 유형인지 확인합니다.
self.y = torch.tensor(self.y).float(): 대상 변수를 PyTorch 텐서로 변환하고 float 유형인지 확인합니다.

6) __len__(self): 이 메소드는 데이터세트의 샘플 수인 데이터세트의 길이를 반환하기 위해 구현됩니다.


7) __getitem__(self, idx): 이 메소드는 데이터 세트의 인덱싱을 허용하도록 구현됩니다. 인덱스 'idx'를 가져와 해당 인덱스에 해당하는 기능 'x'와 타겟 'y'를 반환합니다. 예를 들어 DataLoader를 사용하여 데이터세트를 반복할 때 사용됩니다. 여기에서는 기능 및 대상 텐서에서 idx\번째 행을 검색하여 반환합니다.


따라서 CustomDataset 인스턴스를 생성하면 기능을 표준화하고 이를 PyTorch 텐서로 변환하며 PyTorch의 데이터 세트 처리 유틸리티와의 호환성을 보장하여 PyTorch 모델에 공급할 데이터를 준비합니다.


[__init__ 과 self란?](https://wikidocs.net/192016)

In [ ]:
# Custom으로 정의한 데이터셋 생성
dataset = CustomDataset(df, "PRICE", True)

dataset[10] # getitem이 구현이 되야 사용 가능


Custom으로 정의한 데이터셋은 torch.utils.data.DataLoader에 주입할 수 있습니다.

In [ ]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

batch?  506개 행 묶음으로 학습한다
batch_size 한개의 그룹에 들어가는 데이터의 개수
한개 그룹에 20개씩 묶어서 학습 -> 500개 일시 25번의 입력이 필요
이 전체 데이터를 학습했으니 1 epoch
왜 batch size를 주는가?
하나씩 하면 시간이 너무 오래걸린다

In [ ]:
x, y = next(iter(data_loader))

x.shape, y.shape
# 13 행의 개수 #묶은 행 32 target도 32


# PyTorch를 활용하여 회귀(regression) 예측

In [ ]:
# Device 설정 (cuda:0 혹은 cpu)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#cuda-> gpu를 쓰겠다, 병렬 연산을 하겠다
#cpu는 iq150*4명
#gpu는 50*4000명

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Net - Network, 신경망 클래스 Net 정의
class Net(nn.Module):
    def __init__(self, num_features): # 13개의 feature가 들어간다, Net 클래스를 초기화한다. 입력 특성의 수를 나타내는 num_features를 인수로 사용한다
        super(Net, self).__init__() # 상위 클래스('nn.Module')의 생성자를 호출하여 'Net' 클래스를 신경망 모듈로 초기화한다
        self.fc1 = nn.Linear(num_features, 32) #13 32 32 8 8 1 왼쪽이 Input #num_features 입력 특성과 32개의 출력 특성이 있는 첫 번째 완전 연결(밀집) 레이어를 정의합니다.
        self.fc2 = nn.Linear(32, 8) #fc? fully connected 32개의, 입력 기능(이전 레이어의 출력)과 8개의 출력 기능을 사용하여 두 번째 완전 연결 레이어를 정의합니다
        # 마지막 출력층의 Neuron은 1개로 설정
        self.output = nn.Linear(8, 1)

    def forward(self, x): #신경망의 순방향 전달을 정의합니다.
        x = F.relu(self.fc1(x)) #첫 번째 완전 연결 계층(self.fc1)의 출력에 ReLU(Rectified Linear Unit) 활성화 함수를 적용합니다
        x = F.relu(self.fc2(x)) #두 번째 완전 연결 계층(self.fc2)의 출력에 ReLU 활성화 함수를 적용합니다.
        x = self.output(x) #활성화 함수를 적용하지 않고 신경망의 출력을 계산합니다. 이는 출력이 연속 값인 회귀 작업에 자주 사용됩니다.
        return x #출력 텐서를 반환합니다.

In [ ]:
# 모델 생성
model = Net(NUM_FEATURES)
# 모델을 device 에 올립니다. (cuda:0 혹은 cpu)
model.to(device)
model

# 손실함수(Loss Function) / 옵티마이저(Optimzier) 정의

In [ ]:
# Mean Squared Error(MSE) 오차 정의
loss_fn = nn.MSELoss()

# 옵티마이저 설정: model.paramters()와 learning_rate 설정
optimizer = optim.Adam(model.parameters(), lr=0.005)

# 경사하강법을 활용한 회귀 예측

In [ ]:
# 최대 반복 횟수 정의
num_epoch = 200

# loss 기록하기 위한 list 정의
losses = []

for epoch in range(num_epoch):
    # loss 초기화
    running_loss = 0
    for x, y in data_loader:
        # x, y 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        x = x.to(device)
        y = y.to(device)

        # 그라디언트 초기화 (초기화를 수행하지 않으면 계산된 그라디언트는 누적됩니다.)
        optimizer.zero_grad()

        # output 계산: model의 __call__() 함수 호출
        y_hat = model(x)

        # 손실(loss) 계산
        loss = loss_fn(y.view(-1, 1), y_hat)

        # 미분 계산
        loss.backward()

        # 경사하강법 계산 및 적용
        optimizer.step()

        # 배치별 loss 를 누적합산 합니다.
        running_loss += loss.item()

    # 누적합산된 배치별 loss값을 배치의 개수로 나누어 Epoch당 loss를 산출합니다.
    loss = running_loss / len(data_loader)
    losses.append(loss)

    # 20번의 Epcoh당 출력합니다.
    if epoch % 20 == 0:
        print("{0:05d} loss = {1:.5f}".format(epoch, loss))

print("----" * 15)
print("{0:05d} loss = {1:.5f}".format(epoch, loss))